# OpenAIAgent with Playgrounds Subgraph Connector

In this tutorial, we'll dive into how you can use the OpenAIAgent available through Llama_index in combination with the `PlaygroundsSubgraphConnector` to query data from any decentralized subgraph on the graph network via the Playgrounds API.


## Advantages of this tool:

- **Easy access to Decentralized Subgraphs (Datasets)**: No need for wallet or GRT management.
- **LLM x Blockchain data**: Develop Ai applications that leverage blockchain data seamlessly.
- **Intutive Analytics**: Query blockchain data using natural language.

## Need to know:

The OpenAIAgent allows developers to integrate a diverse range of tools and data sources with OpenAI's language models. When combined with the PlaygroundsSubgraphConnector, you can easily fetch and analyze data from decentralized subgraphs.

Prerequisites:

- An API key from OpenAI: https://platform.openai.com/
- An API key from Playgrounds: https://app.playgrounds.network/signup
- A subgraph identifier or deployment ID : https://thegraph.com/explorer

To learn more about Playgrounds, please visit our website : https://playgrounds.network/


In [3]:
import openai
from llama_index.agent import OpenAIAgent
from base import PlaygroundsSubgraphConnectorToolSpec

def prompt_to_data(prompt, openai_key, identifier, pg_key):
    """
    Query any decentralized subgraph using OpenAIAgent and Playgrounds API.
    
    Args:
        prompt (str): The query text to be used for the GraphQL request.
        openai_key (str): The API key for OpenAI.
        identifier (str): The identifier for the subgraph or deployment.
        pg_key (str): The API key for Playgrounds.
        use_deployment_id (bool): Flag to use deployment id in the URL.
        
    Returns:
        str: Agent's response.
    """
    
    # Set the OpenAI API key
    openai.api_key = openai_key
    
    # Initialize the tool specification
    connector_spec = PlaygroundsSubgraphConnectorToolSpec(
        identifier=identifier, api_key=pg_key)
    
    # Setup agent with the tool
    agent = OpenAIAgent.from_tools(connector_spec.to_tool_list())
    
    # Make a query using the agent
    response = agent.chat(prompt)
    return response


## Simple prompts

For users who are familiar with the structure and content of the subgraph they're working with, the process can be streamlined even further. Instead of crafting extensive GraphQL queries, you can simply describe the data you want in plain English.

In the example below, we're interested in the last 30 days of `financialsDailySnapshots` for specific fields: `timestamp`, `totalValueLockedUSD`, `dailyBorrowUSD`, and `dailyDepositUSD`

By specifying the entities and fields you're interested in, and providing any additional instructions for the agent, you can retrieve the desired data with minimal effort. This approach is especially useful for quick data retrieval tasks or when exploring a familiar subgraph.

Note: Always ensure your prompts are clear and specific to get the most accurate results from the agent.

In [6]:
openai_key = "YOUR_OPENAI_API_KEY"
identifier = "YOUR_SUBGRAPH_OR_DEPLOYMENT_IDENTIFIER"
pg_key = "YOUR_PLAYGROUNDS_API_KEY"

In [7]:
prompt = "Query the first 5 days of financialsDailySnapshots for id, timestamp, totalValueLockedUSD, and dailyVolumeUSD."

print(prompt_to_data(prompt,openai_key,identifier,pg_key))

Here are the first 5 days of financialsDailySnapshots:

1. Snapshot ID: 18751
   - Timestamp: 1620171619
   - Total Value Locked (USD): 101,976.37
   - Daily Volume (USD): 0

2. Snapshot ID: 18752
   - Timestamp: 1620259158
   - Total Value Locked (USD): 128,930,008.01
   - Daily Volume (USD): 14,589,148.89

3. Snapshot ID: 18753
   - Timestamp: 1620345584
   - Total Value Locked (USD): 341,711,935.66
   - Daily Volume (USD): 202,165,514.67

4. Snapshot ID: 18754
   - Timestamp: 1620431997
   - Total Value Locked (USD): 469,265,156.96
   - Daily Volume (USD): 345,756,118.07

5. Snapshot ID: 18755
   - Timestamp: 1620518392
   - Total Value Locked (USD): 532,058,061.10
   - Daily Volume (USD): 605,467,603.78


## Structured Prompts

While simple prompts are effective for quick queries, there might be situations where you need more structure and precision in your queries. Especially if you are not entirely certain about the content or when you want to ensure specific details are captured. In these cases, copying the GraphQL query from tools like the subgraph explorer becomes extremely handy.

In the example below, we've taken a direct GraphQL query from the subgraph explorer and provided it as a prompt, asking for a concise summary of the queried data.

This structured approach ensures that the OpenAIAgent gets all the necessary details, leading to more comprehensive and accurate results. It's especially beneficial when you are exploring unfamiliar subgraphs or want to ensure that specific fields and entities are included in the query.

Note: When copying GraphQL queries, always make sure the syntax is correct and the query is compatible with the target subgraph to avoid errors.

In [8]:
prompt = """
    {
      financialsDailySnapshots(first: 5, orderBy: timestamp, orderDirection: desc) {
        id
        timestamp
        totalValueLockedUSD
        dailyVolumeUSD
        dailyTotalRevenueUSD
        dailySupplySideRevenueUSD
        dailyProtocolSideRevenueUSD
        cumulativeVolumeUSD
        cumulativeTotalRevenueUSD
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        blockNumber
      }
    }
    provide a concise summary of the queried data
    """

print(prompt_to_data(prompt,openai_key,identifier,pg_key))


Here is a concise summary of the queried data:

1. Financial Snapshot 1:
   - ID: 19639
   - Timestamp: 1696885883
   - Total Value Locked (USD): 3,511,000,000,999,999,642,224,684.13
   - Daily Volume (USD): 442,772,734.01
   - Daily Total Revenue (USD): 474,117.24
   - Daily Supply Side Revenue (USD): 474,117.24
   - Daily Protocol Side Revenue (USD): 0
   - Cumulative Volume (USD): 985,774,966,815.87
   - Cumulative Total Revenue (USD): 1,454,908,353.73
   - Cumulative Supply Side Revenue (USD): 1,454,908,353.73
   - Cumulative Protocol Side Revenue (USD): 0
   - Block Number: 18,315,507

2. Financial Snapshot 2:
   - ID: 19638
   - Timestamp: 1696809599
   - Total Value Locked (USD): 3,511,000,000,999,999,685,222,546.80
   - Daily Volume (USD): 356,476,996.90
   - Daily Total Revenue (USD): 346,871.23
   - Daily Supply Side Revenue (USD): 346,871.23
   - Daily Protocol Side Revenue (USD): 0
   - Cumulative Volume (USD): 985,332,194,081.86
   - Cumulative Total Revenue (USD): 1,454,4

## Using with Langchain

The `PlaygroundsSubgraphConnectorToolSpec` can be used with Langchain agents, by converting to a LangChain tool

In this example, we do the following:

- Create structured tools from the `PlaygroundsSubgraphInspectorToolSpec()`.
- Transform the connector tool into a format compatible with LangChain using `to_langchain_tool()`.
- Initialize the LangChain agent with the structured tool, in this case we use `StructuredTool.from_function()`
- Initialize the LangChain agent with `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` for chat output 

In [9]:
import os
import openai
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from base import PlaygroundsSubgraphConnectorToolSpec
from typing import Union

def fetch_from_playgrounds(query) -> Union[dict, str]:
    """
    Fetch data from subgraph via Playgrounds API using the provided GraphQL query.
    
    Args:
        query (str): The GraphQL query string to execute.

    Returns:
        dict: The response from the GraphQL server if successful.
        str: Error message if the request fails.
    """
    
    # Instantiate the connector
    connector = PlaygroundsSubgraphConnectorToolSpec(
        identifier=identifier, 
        api_key=pg_key
        )
    # Use the graphql_request method to fetch data
    response = connector.graphql_request(query)
    return response

# Convert the fetch_from_playgrounds function into a StructuredTool is then used with LangChain
structured_tool = StructuredTool.from_function(fetch_from_playgrounds)

def prompt_to_data_langchain(prompt, openai_key, identifier, pg_key):
    """
    Querying subgraph using LangChain Agent and Playgrounds API.
    """
    
    # Set the OpenAI API key and Initialize the LLM
    openai.api_key = openai_key
    llm = OpenAI(temperature=0)
    
    # Initialize the agent with the previously created structured_tool
    agent_executor = initialize_agent(
        [structured_tool],
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    # Process the query using the LangChain agent
    response = agent_executor.run(prompt)
    return response

In [10]:
prompt = 'Query the first 5 days of financialsDailySnapshots for id, timestamp, totalValueLockedUSD, and dailyVolumeUSD.'

print(prompt_to_data_langchain(prompt,openai_key,identifier,pg_key))



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "fetch_from_playgrounds",
  "action_input": "{ financialsDailySnapshots(first: 5) { id timestamp totalValueLockedUSD dailyVolumeUSD } }"
}
```


Observation: {'data': {'financialsDailySnapshots': [{'id': '18751', 'timestamp': '1620171619', 'totalValueLockedUSD': '101976.3680947069357315540920392817', 'dailyVolumeUSD': '0'}, {'id': '18752', 'timestamp': '1620259158', 'totalValueLockedUSD': '128930008.0119693564848598071496047', 'dailyVolumeUSD': '14589148.89160808212877907657332154'}, {'id': '18753', 'timestamp': '1620345584', 'totalValueLockedUSD': '341711935.66063250446414077793037', 'dailyVolumeUSD': '202165514.672696141748426137929786'}, {'id': '18754', 'timestamp': '1620431997', 'totalValueLockedUSD': '469265156.9558929226412297565582368', 'dailyVolumeUSD': '345756118.0718718857741980736181714'}, {'id': '18755', 'timestamp': '1620518392', 'totalValueLockedUSD': '532058061.0991127647664666899727599', 'dailyVolumeUSD'